In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 5.4 MB/s 
     |████████████████████████████████| 209 kB 52.9 MB/s 
     |████████████████████████████████| 81 kB 4.7 MB/s 
     |████████████████████████████████| 78 kB 4.1 MB/s 
     |████████████████████████████████| 147 kB 77.8 MB/s 
     |████████████████████████████████| 50 kB 2.4 MB/s 
     |████████████████████████████████| 112 kB 41.8 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=fb5939c7bbf30f26628f36fe377043bf3020d0dad4e5cb4cae258c4f7317636a
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip


In [ ]:
import optuna

In [ ]:
#https://medium.com/koderunners/genetic-algorithm-part-3-knapsack-problem-b59035ddd1d6

In [ ]:
import numpy as np
import pandas as pd
import random as rd
from random import randint
import matplotlib.pyplot as plt
item_number = np.arange(1,11)
weight = np.random.randint(1, 15, size = 10)
value = np.random.randint(10, 750, size = 10)
knapsack_threshold = 35    #Maximum weight that the bag of thief can hold 
print('The list is as follows:')
print('Item No.   Weight   Value')
for i in range(item_number.shape[0]):
    print('{0}          {1}         {2}\n'.format(item_number[i], weight[i], value[i]))

The list is as follows:
Item No.   Weight   Value
1          2         745

2          9         347

3          4         441

4          7         60

5          10         712

6          11         272

7          5         714

8          10         190

9          6         562

10          3         182



In [ ]:

# Initilize population each gene index in chromosome indicates whether corresponding item is present or not
#solutions_per_pop = 10
#pop_size = (solutions_per_pop, item_number.shape[0])
#print('Population size = {}'.format(pop_size))
#initial_population = np.random.randint(2, size = pop_size)
#initial_population = initial_population.astype(int)
#num_generations = 50
#print('Initial population: \n{}'.format(initial_population))

# Fitness function ==> fitness is sum of all value of items for each chromosome
def cal_fitness(weight, value, population, threshold, n_survive):
    fitness = np.empty(population.shape[0])
    for i in range(population.shape[0]):
        S1 = np.sum(population[i] * value)
        S2 = np.sum(population[i] * weight)
        if S2 <= threshold:
            fitness[i] = S1
        else :
            fitness[i] = 0 
        #print('prints n_survive', n_survive) 
    survivor_indices = np.argsort(fitness)[:n_survive]
      
      #same number of population members then go to next generation 
    return survivor_indices, fitness[survivor_indices].astype(int)  

# Selection
def selection(fitness, num_parents, population):
    #fitness = list(fitness)
    parents = np.empty((num_parents, population.shape[1]))
    for i in range(num_parents):
        #max_fitness_idx = np.where(fitness == np.argmax(fitness).any())
        max_fitness_idx = np.argmax(fitness)
        parents[i,:] = population[max_fitness_idx, :]
        fitness[max_fitness_idx] = -999999
    return parents

# One-point crossover
def crossover(parents, num_offsprings,crossover_rate):
    offsprings = np.empty((num_offsprings, parents.shape[1]))
    crossover_point = int(parents.shape[1]/2)
    #crossover_rate = 0.4
    i=0
    while (parents.shape[0] < num_offsprings):
        parent1_index = i%parents.shape[0]
        parent2_index = (i+1)%parents.shape[0]
        x = rd.random()
        if x > crossover_rate:
            continue
        parent1_index = i%parents.shape[0]
        parent2_index = (i+1)%parents.shape[0]
        offsprings[i,0:crossover_point] = parents[parent1_index,0:crossover_point]
        offsprings[i,crossover_point:] = parents[parent2_index,crossover_point:]
        i=+1
    return offsprings    

# Random mutation
def mutation(offsprings,mutation_rate):
    mutants = np.empty((offsprings.shape))
    #mutation_rate = 0.5
    for i in range(mutants.shape[0]):
        random_value = rd.random()
        mutants[i,:] = offsprings[i,:]
        if random_value > mutation_rate:
            continue
        int_random_value = randint(0,offsprings.shape[1]-1)    
        if mutants[i,int_random_value] == 0 :
            mutants[i,int_random_value] = 1
        else :
            mutants[i,int_random_value] = 0
    return mutants   

def objective(trial=None):
    global item_number

    #def __init__(self,solutions_per_pop):

    solutions_per_pop = 10
    pop_size = (solutions_per_pop, item_number.shape[0])
    # print('Population size = {}'.format(pop_size))
    initial_population = np.random.randint(2, size = pop_size)
    initial_population = initial_population.astype(int)
    num_generations = 50
    # print('Initial population: \n{}'.format(initial_population))
    threshold = 35 
    parameters, fitness_history = [], []

    if trial is not None:
        parameters, fitness_history = [], []
        solutions_per_pop = trial.suggest_int('solutions_per_pop',4,16)
        
        # print('solutions_per_pop', solutions_per_pop)
        crossover_rate = trial.suggest_float('crossover_rate',0.1, 1) 
        # print('crossover rate', crossover_rate)
        mutation_rate = trial.suggest_float('mutation_rate',0.1,0.5)
        # print('mutation rate', mutation_rate)
        pop_size = (solutions_per_pop, item_number.shape[0])
        # print('Population size = {}'.format(pop_size))
        #initial_population = np.random.randint(2, size = pop_size)
        #initial_population = initial_population.astype(int)
    else:
        solutions_per_pop = 14
        # print('solutions_per_pop', solutions_per_pop)
        crossover_rate = 0.15734363786505476
        # print('crossover rate', crossover_rate)
        mutation_rate = 0.18866446478557403
        # print('mutation rate', mutation_rate)
        pop_size = (solutions_per_pop, item_number.shape[0])
        # print('Population size = {}'.format(pop_size))
        #initial_population = np.random.randint(2, size = pop_size)
        #initial_population = initial_population.astype(int)

    num_generations = 50
    num_parents = int(pop_size[0]/2)
    num_offsprings = pop_size[0] - num_parents
    population = initial_population
    survival_indices, fitness = cal_fitness(weight, value, population, threshold, solutions_per_pop)
    population = population[survival_indices]
    for i in range(num_generations):
        fitness = cal_fitness(weight, value, population, threshold, solutions_per_pop)
        fitness_history.append(fitness)
        parents = selection(fitness, num_parents, population)
        offsprings = crossover(parents, num_offsprings,crossover_rate)
        mutants = mutation(offsprings,mutation_rate)
        #print(parents.shape)
        #print(mutants.shape)
        #print(offsprings.shape)
        #print(population.shape)
        #population[0:parents.shape[0], :] = parents
       # population[parents.shape[0]:, :] = mutants
        population = np.concatenate((parents,mutants))
        # print('parents', parents)
       # n_survive = 5
        #n_survive = pop_size[0]
        
        survivor_indices, fitness = cal_fitness(weight, value, population, threshold, solutions_per_pop)
        population = population[survivor_indices]
    #parameters and fitness of solutions
    # print('Last generation: \n{}\n'.format(population)) 
    fitness_last_gen = cal_fitness(weight, value, population, threshold,solutions_per_pop)      
    # print('Fitness of the last generation: \n{}\n'.format(fitness_last_gen))   
    max_fitness = np.max(fitness_last_gen)
    print('max_fitness',max_fitness)
    parameters.append(population[max_fitness[0][0],:])
    return parameters, fitness_history
    return max_fitness

'''
    #print which items should be selected after consideration of weight etc..
    #parameters, fitness_history = objective(weight, value, initial_population, pop_size, num_generations, knapsack_threshold)
    #print('The optimized parameters for the given inputs are: \n{}'.format(parameters))
    #selected_items = item_number * parameters
   # print('\nSelected items that will maximize the knapsack without breaking it:')
    #for i in range(selected_items.shape[1]):
      #  if selected_items[0][i] != 0:
        #    print('{}\n'.format(selected_items[0][i]))

    #plot best fitness of best solutions through generations 
    
    fitness_history_mean = [np.mean(fitness) for fitness in fitness_history]
    fitness_history_max = [np.max(fitness) for fitness in fitness_history]
    plt.plot(list(range(num_generations)), fitness_history_mean, label = 'Mean Fitness')
    plt.plot(list(range(num_generations)), fitness_history_max, label = 'Max Fitness')
    plt.legend()
    plt.title('Fitness through the generations')
    plt.xlabel('Generations')
    plt.ylabel('Fitness')
    plt.show()
    print(np.asarray(fitness_history).shape)

  '''


"\n    #print which items should be selected after consideration of weight etc..\n    #parameters, fitness_history = objective(weight, value, initial_population, pop_size, num_generations, knapsack_threshold)\n    #print('The optimized parameters for the given inputs are: \n{}'.format(parameters))\n    #selected_items = item_number * parameters\n   # print('\nSelected items that will maximize the knapsack without breaking it:')\n    #for i in range(selected_items.shape[1]):\n      #  if selected_items[0][i] != 0:\n        #    print('{}\n'.format(selected_items[0][i]))\n\n    #plot best fitness of best solutions through generations \n    \n    fitness_history_mean = [np.mean(fitness) for fitness in fitness_history]\n    fitness_history_max = [np.max(fitness) for fitness in fitness_history]\n    plt.plot(list(range(num_generations)), fitness_history_mean, label = 'Mean Fitness')\n    plt.plot(list(range(num_generations)), fitness_history_max, label = 'Max Fitness')\n    plt.legend()\n  

In [ ]:
objective(trial=None)

IndexError: ignored

In [ ]:
#from functools import partial
print(optuna.logging.get_verbosity()) 
print(optuna.logging.INFO)

study = optuna.create_study(direction='maximize')

study.optimize(objective, n_trials=5)


[I 2022-12-12 18:41:32,678] A new study created in memory with name: no-name-c7b1a75f-cc03-4081-bbb2-9876666befe2
[W 2022-12-12 18:41:32,684] Trial 0 failed because of the following error: IndexError('index 19 is out of bounds for axis 0 with size 10')
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-6-fd8569d51767>", line 121, in objective
    parents = selection(fitness, num_parents, population)
  File "<ipython-input-6-fd8569d51767>", line 33, in selection
    parents[i,:] = population[max_fitness_idx, :]
IndexError: index 19 is out of bounds for axis 0 with size 10


20
20


IndexError: ignored